1. Swap 4 to 1 in target column
2. convert all text into lower case
3. remove username
4. Removing Numbers
5. removing punctuation
7. removing urls
6. removing extra space
8. tokenize the text into words
9. remove the stop words
10. stemming the words

In [161]:
import re
import string
import pandas as pd
from pathlib import Path
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sentimentAnalyzer.utils.common import read_dataset
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [214]:
def lower_case_converter(text):
    return text.lower()

def basic_remove_process(text):
    # Remove users
    no_user_text = re.sub(r'@\w+', '', text)
    # Remove Numbers
    remove_number_text = re.sub(r'\d+', '', no_user_text)
    # Remove Punctuation
    translator = str.maketrans('', '', string.punctuation)
    remove_punc_text = remove_number_text.translate(translator)
    # Remove URLs
    remove_url_text = re.sub(r'http\S+|www\S+', '', remove_punc_text)
    # Remove Extra Space
    final_text = " ".join(remove_url_text.split())

    return final_text

def text_tokenizer(text):
    tokenize_lst_text = word_tokenize(text)
    return tokenize_lst_text

def stop_word_removal(text_lst):
    STOPWORD =  set(stopwords.words("english"))
    stopword_remove_text = [word for word in text_lst if word not in STOPWORD]
    return stopword_remove_text

def text_stemmer(text_lst):
    stemmer = SnowballStemmer('english')
    stem_text = [stemmer.stem(word) for word in text_lst]
    return stem_text

def train_text_transformer():
    train_df = read_dataset(Path(config.data_transformation.train_data_file), encoding=parmas.dataset.data_encoding)
    train_df = train_df[["target", "text"]]

    train_df['target'] = train_df['target'].replace(4, 1)

    train_df['text'] = train_df['text'].apply(lambda x: lower_case_converter(x))
    train_df['text'] = train_df['text'].apply(lambda x: basic_remove_process(x))
    train_df['text'] = train_df['text'].apply(lambda x: text_tokenizer(x))
    train_df['text'] = train_df['text'].apply(lambda x: stop_word_removal(x))
    train_df['text'] = train_df['text'].apply(lambda x: text_stemmer(x))

    train_df['text'] = train_df['text'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

    return train_df

In [212]:
def test_text_transformer():
    test_df = read_dataset(Path(config.data_transformation.test_data_file), encoding=parmas.dataset.data_encoding)
    test_df = test_df[["target", "text"]]

    test_df['target'] = test_df['target'].replace(4, 1)

    test_df['text'] = test_df['text'].apply(lambda x: lower_case_converter(x))
    test_df['text'] = test_df['text'].apply(lambda x: basic_remove_process(x))
    test_df['text'] = test_df['text'].apply(lambda x: text_tokenizer(x))
    test_df['text'] = test_df['text'].apply(lambda x: stop_word_removal(x))
    test_df['text'] = test_df['text'].apply(lambda x: text_stemmer(x))

    test_df['text'] = test_df['text'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)   

    return test_df

In [213]:
df_test = test_text_transformer()

[2025-01-03 15:25:29,497: INFO: common: Reading dataset from path: artifacts\data_ingestion\test\test.csv]
[2025-01-03 15:25:31,446: INFO: common: Dataset loaded successfully with 320000 rows and 6 columns.]


In [215]:
df_test.head()

,target,text
0,0,ahhh hope ok
1,0,cool tweet app razr
2,0,know famili drama lamehey next time u hang kim...
3,0,school email wont open geographi stuff revis s...
4,0,upper airway problem


In [164]:
df_test['target'] = df_test['target'].replace(4, 1)
df_test['target'].unique()

array([0, 1])

In [174]:
X_test = df_test['text'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
y_test = df_test['target']

In [154]:
df = train_text_transformer()

[2025-01-03 02:37:05,450: INFO: common: Reading dataset from path: artifacts\data_ingestion\train\train.csv]
[2025-01-03 02:37:09,765: INFO: common: Dataset loaded successfully with 1280000 rows and 6 columns.]


In [156]:
df['target'] = df['target'].replace(4, 1)
df['target'].unique()

array([1, 0])

In [170]:
X_train = df['text'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
y_train = df['target']

In [171]:
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=300000)
vectoriser.fit(X_train)
print("No. of Feature_words: ", len(vectoriser.get_feature_names_out()))

No. of Feature_words:  300000


In [231]:
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=300000)
vectoriser.fit(X_train)
X_train = vectoriser.transform(X_train)
X_test = vectoriser.transform(X_test)

AttributeError: 'csr_matrix' object has no attribute 'lower'

In [172]:
X_train = vectoriser.transform(X_train)

In [177]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 12996765 stored elements and shape (1280000, 300000)>
  Coords	Values
  (0, 21927)	0.18834560650948634
  (0, 38994)	0.2266529703014415
  (0, 97600)	0.09953069629947216
  (0, 97667)	0.3033362119954497
  (0, 140496)	0.10172579504667327
  (0, 141829)	0.33359089799648034
  (0, 187613)	0.22462141622920775
  (0, 187618)	0.244774576110176
  (0, 198100)	0.22288007501087348
  (0, 198226)	0.31102411999397145
  (0, 202841)	0.31784687670853956
  (0, 205118)	0.14660561575350095
  (0, 233354)	0.14906717790505142
  (0, 233427)	0.2061554340305347
  (0, 261074)	0.3375114115145163
  (0, 293832)	0.15596546687088778
  (0, 296255)	0.14099012189396565
  (0, 296652)	0.29574464121558597
  (1, 2340)	0.25944224336969496
  (1, 2437)	0.49262739704644987
  (1, 65739)	0.3490510347355771
  (1, 69504)	0.47757585145198855
  (1, 78091)	0.2809553359383544
  (1, 78102)	0.4562173683488655
  (1, 218813)	0.23026607070750216
  :	:
  (1279998, 28843)	0.213739092837

In [ ]:
X_test = vectoriser.transform(X_test)

In [180]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3183520 stored elements and shape (320000, 300000)>
  Coords	Values
  (0, 3487)	0.38802061594204845
  (0, 3517)	0.6670862598470247
  (0, 116329)	0.23914869327494115
  (0, 116802)	0.5106112743989338
  (0, 181559)	0.2941427416898907
  (1, 11142)	0.299545373022814
  (1, 46898)	0.23186242926404135
  (1, 47240)	0.5083842036366177
  (1, 204667)	0.505148857805784
  (1, 264806)	0.21724585971007923
  (1, 264833)	0.5437708642300568
  (2, 34053)	0.15125823363111143
  (2, 62890)	0.2208342430190118
  (2, 75320)	0.16822707725044758
  (2, 75374)	0.341832834380506
  (2, 103245)	0.14462932659491592
  (2, 103517)	0.2631356377436169
  (2, 106090)	0.17580805983010853
  (2, 121609)	0.13139796657418784
  (2, 121681)	0.26165086841161433
  (2, 133168)	0.24217208660440823
  (2, 134231)	0.11748648132058323
  (2, 134519)	0.324934322030119
  (2, 140496)	0.10627536619059096
  (2, 175407)	0.14555629701945746
  :	:
  (319996, 281461)	0.2331920895350605
  

In [197]:
import os
from pathlib import Path

In [200]:
a = Path("artifacts/data_transformation/train/X_train.npz")

In [201]:
a.suffix

'.npz'

In [208]:
from scipy.sparse import save_npz, load_npz
import numpy as np
from sentimentAnalyzer.logging import logger
from sentimentAnalyzer.utils.common import DataInfo

ImportError: cannot import name 'DataInfo' from 'sentimentAnalyzer.utils.common' (D:\CDAC\Machine Learning\sentiment-analyzer\src\sentimentAnalyzer\utils\common.py)

In [210]:
a = DataInfo()

NameError: name 'DataInfo' is not defined

In [ ]:
def save_transformed_data_file(path: Path, data, data_info: DataInfo) -> None:
    if path.suffix == ".npz":
        save_npz(path, data)
        logger.info(f"Input Data is Saved in {path}")
    elif path.suffix == ".npy":
        np.save(path, data)
        logger.info(f"Output Data is Saved in {path}")
    else:
        logger.info(f"{path.suffix} is not a correct extension {data_info} will not be stored")
    
    return None

In [186]:
from scipy.sparse import save_npz, load_npz
import numpy as np

save_npz("X_train.npz", X_train)
np.save("y_train.npy", y_train)

In [185]:
save_npz("X_test.npz", X_train)
np.save("y_test.npy", y_train)

In [187]:
X_train_loaded = load_npz("X_train.npz")
y_train_loaded = np.load("y_train.npy")

In [192]:
X_test_loaded = load_npz("X_test.npz")
y_test_loaded = np.load("y_test.npy")

In [191]:
y_train_loaded

array([1, 1, 1, ..., 0, 0, 0], shape=(1280000,))

In [190]:
print(X_train_loaded.shape)

(1280000, 300000)


In [193]:
def model_evaluation(model):

    y_pred = model.predict(X_test_loaded)

    print(classification_report(y_test_loaded, y_pred))

    cf_matrix = confusion_matrix(y_test_loaded, y_pred)

    print(cf_matrix)

In [194]:
from sklearn.linear_model import LogisticRegression
lrmodel = LogisticRegression(C = 2, max_iter=1000, n_jobs=-1)
lrmodel.fit(X_train_loaded, y_train_loaded)
model_evaluation(lrmodel)

              precision    recall  f1-score   support

           0       0.84      0.82      0.83    640506
           1       0.82      0.85      0.84    639494

    accuracy                           0.83   1280000
   macro avg       0.83      0.83      0.83   1280000
weighted avg       0.83      0.83      0.83   1280000

[[524120 116386]
 [ 96995 542499]]


In [137]:
data = df[['target', 'text']]
data['target'] = data['target'].replace(4, 1)
# print(data['target'].unique())

# Seperating the Positive and Negative Tweet Data
data_pos = data[data['target'] == 1]
data_neg = data[data['target'] == 0]
# print(data_neg,data_pos)

# Taking Less values for calculation purpose
data_pos = data_pos.iloc[:int(20000)]
data_neg = data_neg.iloc[:int(20000)]

# Making text in lower case for consistency

dataset = pd.concat([data_pos, data_neg])

C:\Users\ehtes\AppData\Local\Temp\ipykernel_16636\4249706494.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target'] = data['target'].replace(4, 1)


In [144]:
data.head()

,target,text
0,1,"@jbtaylor WIth ya. &quot;I'd like a Palm Pre, ..."
1,1,"felt the earthquake this afternoon, it seems t..."
2,1,"Ruffles on shirts are like so in, me Likey"
3,0,Pretty bad night into a crappy morning....FML!...
4,1,"@dcbriccetti yeah, what a clear view!"


In [141]:
dataset["target"][0] = 0

C:\Users\ehtes\AppData\Local\Temp\ipykernel_16636\3612197400.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataset["target"][0] = 0
C:\Users\ehtes\AppData\Local\Temp\ipykernel_16636\3612197400.py:1: SettingWithCopyWarning: 
A value is 

In [142]:
dataset.head()

,target,text
0,0,"@jbtaylor WIth ya. &quot;I'd like a Palm Pre, ..."
1,1,"felt the earthquake this afternoon, it seems t..."
2,1,"Ruffles on shirts are like so in, me Likey"
4,1,"@dcbriccetti yeah, what a clear view!"
5,1,One more time Follow my fam #FF @georgeann13 @...


In [112]:
text = df['text'][0]+"123"
text

"@jbtaylor WIth ya. &quot;I'd like a Palm Pre, Touchstone charger. ReadyNow? Yes, that sounds good. But is my beer ready now?'  #prelaunch123"

In [113]:
lower_text = text.lower()
lower_text

"@jbtaylor with ya. &quot;i'd like a palm pre, touchstone charger. readynow? yes, that sounds good. but is my beer ready now?'  #prelaunch123"

In [114]:
no_user_text = re.sub(r'@\w+', '', lower_text)
no_user_text

" with ya. &quot;i'd like a palm pre, touchstone charger. readynow? yes, that sounds good. but is my beer ready now?'  #prelaunch123"

In [115]:
remove_number_text = re.sub(r'\d+', '', no_user_text)
remove_number_text

" with ya. &quot;i'd like a palm pre, touchstone charger. readynow? yes, that sounds good. but is my beer ready now?'  #prelaunch"

In [117]:
translator = str.maketrans('', '', string.punctuation)
remove_punc_text = remove_number_text.translate(translator)
remove_punc_text

' with ya quotid like a palm pre touchstone charger readynow yes that sounds good but is my beer ready now  prelaunch'

In [118]:
remove_space_text = " ".join(remove_punc_text.split())
remove_space_text

'with ya quotid like a palm pre touchstone charger readynow yes that sounds good but is my beer ready now prelaunch'

In [119]:
remove_url_text = re.sub(r'http\S+|www\S+', '', remove_space_text)
remove_url_text

'with ya quotid like a palm pre touchstone charger readynow yes that sounds good but is my beer ready now prelaunch'

In [121]:
tokenize_text = word_tokenize(remove_url_text)
tokenize_text

['with',
 'ya',
 'quotid',
 'like',
 'a',
 'palm',
 'pre',
 'touchstone',
 'charger',
 'readynow',
 'yes',
 'that',
 'sounds',
 'good',
 'but',
 'is',
 'my',
 'beer',
 'ready',
 'now',
 'prelaunch']

In [123]:
STOPWORD =  set(stopwords.words("english"))
stopword_remove_text = [word for word in tokenize_text if word not in STOPWORD]
stopword_remove_text

['ya',
 'quotid',
 'like',
 'palm',
 'pre',
 'touchstone',
 'charger',
 'readynow',
 'yes',
 'sounds',
 'good',
 'beer',
 'ready',
 'prelaunch']

In [128]:
stemmer = SnowballStemmer('english')
stem_text = [stemmer.stem(word) for word in stopword_remove_text]
stem_text

['ya',
 'quotid',
 'like',
 'palm',
 'pre',
 'touchston',
 'charger',
 'readynow',
 'yes',
 'sound',
 'good',
 'beer',
 'readi',
 'prelaunch']

In [ ]:
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=500000)

In [1]:
import os

In [2]:
%pwd

'd:\\CDAC\\Machine Learning\\sentiment-analyzer\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\CDAC\\Machine Learning\\sentiment-analyzer'

In [12]:
from dataclasses import dataclass
from pathlib import Path

In [13]:
@dataclass(frozen=True)
class DataTransformationConfig:
    dataset_dir: Path
    transformed_data_dir: Path
    train_data_file: Path
    test_data_file: Path
    train_transformed_dir: Path
    test_transformed_dir: Path
    encoder: str

In [14]:
from sentimentAnalyzer.constant import *
from sentimentAnalyzer.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        dataset = self.params.dataset
        create_directories([config.transformed_data_dir])
        create_directories([config.train_transformed_dir])
        create_directories([config.test_transformed_dir])

        data_transformation_config = DataTransformationConfig(
            dataset_dir = Path(config.dataset_dir),
            transformed_data_dir = Path(config.transformed_data_dir),
            train_data_file = Path(config.train_data_file),
            test_data_file = Path(config.test_data_file),
            train_transformed_dir = Path(config.train_transformed_dir),
            test_transformed_dir = Path(config.test_transformed_dir),
            encoder = dataset.data_encoding
        )

        return data_transformation_config

In [16]:
import os
import re
import string
from sentimentAnalyzer.logging import logger
from sentimentAnalyzer.utils.common import read_dataset, save_transformed_data_file, DataInfo
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [17]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def lower_case_converter(self, text):
        return text.lower()

    def basic_remove_process(self, text):
        # Remove users
        no_user_text = re.sub(r'@\w+', '', text)
        # Remove Numbers
        remove_number_text = re.sub(r'\d+', '', no_user_text)
        # Remove Punctuation
        translator = str.maketrans('', '', string.punctuation)
        remove_punc_text = remove_number_text.translate(translator)
        # Remove URLs
        remove_url_text = re.sub(r'http\S+|www\S+', '', remove_punc_text)
        # Remove Extra Space
        final_text = " ".join(remove_url_text.split())

        return final_text

    def text_tokenizer(self, text):
        tokenize_lst_text = word_tokenize(text)
        return tokenize_lst_text

    def stop_word_removal(self, text_lst):
        STOPWORD =  set(stopwords.words("english"))
        stopword_remove_text = [word for word in text_lst if word not in STOPWORD]
        return stopword_remove_text

    def text_stemmer(self, text_lst):
        stemmer = SnowballStemmer('english')
        stem_text = [stemmer.stem(word) for word in text_lst]
        return stem_text
    
    def text_transformer(self, path: Path, data_info: DataInfo):
        df = read_dataset(path, encoding=self.config.encoder)
        df = df[["target", "text"]]

        df['target'] = df['target'].replace(4, 1)

        df['text'] = df['text'].apply(lambda x: self.lower_case_converter(x))
        df['text'] = df['text'].apply(lambda x: self.basic_remove_process(x))
        df['text'] = df['text'].apply(lambda x: self.text_tokenizer(x))
        df['text'] = df['text'].apply(lambda x: self.stop_word_removal(x))
        df['text'] = df['text'].apply(lambda x: self.text_stemmer(x))

        df['text'] = df['text'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
        logger.info(f"{data_info.value} has been Pre-Processed")

        return df['text'], df['target']
    

    def data_transformation(self) -> None:
        X_train, y_train = self.text_transformer(path=self.config.train_data_file, data_info=DataInfo.TRAINING)
        X_test, y_test = self.text_transformer(path=self.config.test_data_file, data_info=DataInfo.TESTING)
        
        vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=300000)
        vectoriser.fit(X_train)
        X_train = vectoriser.transform(X_train)
        X_test = vectoriser.transform(X_test)

        # Saving the Data
        # Training Data
        x_tr_path = Path(f"{self.config.train_transformed_dir}/X_train.npz")
        y_tr_path = Path(f"{self.config.train_transformed_dir}/y_train.npy")
        save_transformed_data_file(x_tr_path, X_train, data_info=DataInfo.TRAINING)
        save_transformed_data_file(y_tr_path, y_train, data_info=DataInfo.TRAINING)
        # Testing Day
        x_test_path = Path(f"{self.config.test_transformed_dir}/X_test.npz")
        y_test_path = Path(f"{self.config.test_transformed_dir}/y_test.npy")
        save_transformed_data_file(x_test_path, X_test, data_info=DataInfo.TESTING)
        save_transformed_data_file(y_test_path, y_test, data_info=DataInfo.TESTING)
        return None
    

In [18]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_validation = DataTransformation(config=data_transformation_config)
    data_validation.data_transformation()
except Exception as e:
    raise e

[2025-01-03 20:20:34,451: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-03 20:20:34,453: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-03 20:20:34,454: INFO: common: created directory at: artifacts]
[2025-01-03 20:20:34,456: INFO: common: created directory at: artifacts/data_transformation]
[2025-01-03 20:20:34,457: INFO: common: created directory at: artifacts/data_transformation/train]
[2025-01-03 20:20:34,458: INFO: common: created directory at: artifacts/data_transformation/test]
[2025-01-03 20:20:34,459: INFO: common: Reading dataset from path: artifacts\data_ingestion\train\train.csv]
[2025-01-03 20:20:38,523: INFO: common: Dataset loaded successfully with 1280000 rows and 6 columns.]
[2025-01-03 20:25:40,423: INFO: 3014423729: Transformed Training Data has been Pre-Processed]
[2025-01-03 20:25:40,423: INFO: common: Reading dataset from path: artifacts\data_ingestion\test\test.csv]
[2025-01-03 20:25:41,238: INFO: common: Dataset lo

In [5]:
from sentimentAnalyzer.utils.common import load_transformed_data_file, DataInfo
from pathlib import Path

In [22]:
X_train = load_transformed_data_file(Path(f"artifacts/data_transformation/train/X_train.npz"), data_info=DataInfo.TRAINING)
y_train = load_transformed_data_file(Path(f"artifacts/data_transformation/train/y_train.npy"), data_info=DataInfo.TRAINING)

[2025-01-03 20:45:52,910: INFO: common: Transformed Training Data is load from artifacts\data_transformation\train\X_train.npz]
[2025-01-03 20:45:52,921: INFO: common: Transformed Training Data is load from artifacts\data_transformation\train\y_train.npy]


In [23]:
X_test = load_transformed_data_file(Path(f"artifacts/data_transformation/test/X_test.npz"), data_info=DataInfo.TESTING)
y_test = load_transformed_data_file(Path(f"artifacts/data_transformation/test/y_test.npy"), data_info=DataInfo.TESTING)

[2025-01-03 20:45:53,618: INFO: common: Transformed Testing Data is load from artifacts\data_transformation\test\X_test.npz]
[2025-01-03 20:45:53,627: INFO: common: Transformed Testing Data is load from artifacts\data_transformation\test\y_test.npy]


In [27]:
Path(f"artifacts/data_transformation/test/X_test.npz").exists()

True

In [8]:
type(y_data)

numpy.ndarray

In [9]:
type(tr_data)

scipy.sparse._csr.csr_matrix

In [24]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression

In [25]:
def model_evaluation(model):

    y_pred = model.predict(X_test)

    print(classification_report(y_test, y_pred))

    cf_matrix = confusion_matrix(y_test, y_pred)

    print(cf_matrix)

In [26]:
lrmodel = LogisticRegression(C = 2, max_iter=1000, n_jobs=-1)
lrmodel.fit(X_train, y_train)
model_evaluation(lrmodel)

              precision    recall  f1-score   support

           0       0.80      0.78      0.79    159494
           1       0.79      0.81      0.80    160506

    accuracy                           0.80    320000
   macro avg       0.80      0.80      0.80    320000
weighted avg       0.80      0.80      0.80    320000

[[124195  35299]
 [ 30173 130333]]


In [12]:
print(config, end=",")

{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_dir': 'raw.zip', 'unzip_dir': 'artifacts/data_ingestion', 'dataset_file': 'artifacts/data_ingestion/raw.csv', 'train_data_dir': 'artifacts/data_ingestion/train', 'test_data_dir': 'artifacts/data_ingestion/test'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'STATUS_FILE': 'artifacts/data_validation/status.txt', 'file_check_dir': 'artifacts/data_ingestion', 'FILE_NAMES': ['test', 'train']}, 'data_transformation': {'dataset_dir': 'artifacts/data_ingestion', 'transformed_data_dir': 'artifacts/data_transformation'}},

In [13]:
parmas

ConfigBox({'dataset': {'data_encoding': 'ISO-8859-1', 'train_size': 0.8, 'test_size': 0.2}})

In [14]:
data_trns = config.data_transformation

In [15]:
data_trns

ConfigBox({'dataset_dir': 'artifacts/data_ingestion', 'transformed_data_dir': 'artifacts/data_transformation'})

In [16]:
train_data_file = Path(os.path.join(data_trns.dataset_dir, "train/train.csv"))

In [17]:
train_data_file

WindowsPath('artifacts/data_ingestion/train/train.csv')

In [18]:
import pandas as pd
from sentimentAnalyzer.utils.common import read_dataset

In [110]:
df = read_dataset(path=train_data_file, encoding=parmas.dataset.data_encoding, training=False)

[2025-01-03 01:18:55,775: INFO: common: Reading dataset from path: artifacts\data_ingestion\train\train.csv]
[2025-01-03 01:18:59,894: INFO: common: Dataset loaded successfully with 1280000 rows and 6 columns.]


In [20]:
df.head()

,target,ids,date,flag,user,text
0,4,2051457557,Fri Jun 05 22:04:23 PDT 2009,NO_QUERY,JGoldsborough,"@jbtaylor WIth ya. &quot;I'd like a Palm Pre, ..."
1,4,2053083567,Sat Jun 06 03:12:21 PDT 2009,NO_QUERY,Psioui,"felt the earthquake this afternoon, it seems t..."
2,4,1976779404,Sat May 30 19:02:49 PDT 2009,NO_QUERY,adriville,"Ruffles on shirts are like so in, me Likey"
3,0,2325739990,Thu Jun 25 05:59:18 PDT 2009,NO_QUERY,Blondie128,Pretty bad night into a crappy morning....FML!...
4,4,1973503391,Sat May 30 11:16:35 PDT 2009,NO_QUERY,khrabrov,"@dcbriccetti yeah, what a clear view!"


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1280000 entries, 0 to 1279999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1280000 non-null  int64 
 1   ids     1280000 non-null  int64 
 2   date    1280000 non-null  object
 3   flag    1280000 non-null  object
 4   user    1280000 non-null  object
 5   text    1280000 non-null  object
dtypes: int64(2), object(4)
memory usage: 58.6+ MB


In [37]:
# replacing 4 to 1 in target 
df['target'] = df['target'].replace(4, 1)
df['target'].unique()

array([1, 0])

In [38]:
df.head()

,target,ids,date,flag,user,text
0,1,2051457557,Fri Jun 05 22:04:23 PDT 2009,NO_QUERY,JGoldsborough,"@jbtaylor WIth ya. &quot;I'd like a Palm Pre, ..."
1,1,2053083567,Sat Jun 06 03:12:21 PDT 2009,NO_QUERY,Psioui,"felt the earthquake this afternoon, it seems t..."
2,1,1976779404,Sat May 30 19:02:49 PDT 2009,NO_QUERY,adriville,"Ruffles on shirts are like so in, me Likey"
3,0,2325739990,Thu Jun 25 05:59:18 PDT 2009,NO_QUERY,Blondie128,Pretty bad night into a crappy morning....FML!...
4,1,1973503391,Sat May 30 11:16:35 PDT 2009,NO_QUERY,khrabrov,"@dcbriccetti yeah, what a clear view!"


In [39]:
# changing to lower case of text columns

df['text'] = df['text'].apply(lambda x: x.lower())
df.head()

,target,ids,date,flag,user,text
0,1,2051457557,Fri Jun 05 22:04:23 PDT 2009,NO_QUERY,JGoldsborough,"@jbtaylor with ya. &quot;i'd like a palm pre, ..."
1,1,2053083567,Sat Jun 06 03:12:21 PDT 2009,NO_QUERY,Psioui,"felt the earthquake this afternoon, it seems t..."
2,1,1976779404,Sat May 30 19:02:49 PDT 2009,NO_QUERY,adriville,"ruffles on shirts are like so in, me likey"
3,0,2325739990,Thu Jun 25 05:59:18 PDT 2009,NO_QUERY,Blondie128,pretty bad night into a crappy morning....fml!...
4,1,1973503391,Sat May 30 11:16:35 PDT 2009,NO_QUERY,khrabrov,"@dcbriccetti yeah, what a clear view!"


In [60]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords', download_dir=os.getcwd())
stopwords.words("english")

[nltk_data] Downloading package stopwords to d:\CDAC\Machine
[nltk_data]     Learning\sentiment-analyzer...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [43]:
print(len(stopwords.words('english')))

179


In [45]:
STOPWORD = set(stopwords.words('english'))

In [47]:
from nltk.tokenize import word_tokenize

In [49]:
df['text'][0]

"@jbtaylor with ya. &quot;i'd like a palm pre, touchstone charger. readynow? yes, that sounds good. but is my beer ready now?'  #prelaunch"

In [53]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ehtes\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [70]:
tryout = word_tokenize(df['text'][0])

In [71]:
tryout

['@',
 'jbtaylor',
 'with',
 'ya',
 '.',
 '&',
 'quot',
 ';',
 'i',
 "'d",
 'like',
 'a',
 'palm',
 'pre',
 ',',
 'touchstone',
 'charger',
 '.',
 'readynow',
 '?',
 'yes',
 ',',
 'that',
 'sounds',
 'good',
 '.',
 'but',
 'is',
 'my',
 'beer',
 'ready',
 'now',
 '?',
 "'",
 '#',
 'prelaunch']

In [66]:
stop_try = "".join([word for word in df['text'][0].split() if word not in STOPWORD])

In [67]:
stop_try

"@jbtaylorya.&quot;i'dlikepalmpre,touchstonecharger.readynow?yes,soundsgood.beerreadynow?'#prelaunch"

In [84]:
stop_try = " ".join([word for word in tryout if word not in STOPWORD])

In [85]:
stop_try

"@ jbtaylor ya . & quot ; 'd like palm pre , touchstone charger . readynow ? yes , sounds good . beer ready ? ' # prelaunch"

In [74]:
import string

In [75]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [77]:
clean_data = " ".join([word for word in tryout if word not in string.punctuation])

In [78]:
clean_data

"jbtaylor with ya quot i 'd like a palm pre touchstone charger readynow yes that sounds good but is my beer ready now prelaunch"

In [82]:
"'d" not in string.punctuation

True

In [88]:
translator = str.maketrans('', '', string.punctuation)
clean_data = stop_try.translate(translator)

In [89]:
clean_data

' jbtaylor ya   quot  d like palm pre  touchstone charger  readynow  yes  sounds good  beer ready    prelaunch'

In [90]:
import re

In [95]:
c = re.sub(r'(.)1+', r'1', clean_data)

In [96]:
" ".join(c)

'  j b t a y l o r   y a       q u o t     d   l i k e   p a l m   p r e     t o u c h s t o n e   c h a r g e r     r e a d y n o w     y e s     s o u n d s   g o o d     b e e r   r e a d y         p r e l a u n c h'

# Insight of raw Dataset

In [30]:
df_raw = pd.read_csv("artifacts/data_ingestion/raw.csv", encoding=parmas.dataset.data_encoding,
                     names=["target", "ids", "date", "flag", "user", "text"])

In [31]:
df_raw.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [32]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   ids     1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB
